# Sample Dataset Generator

In [1]:
# Display all rows in pandas dataframes
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
print("✅ Display settings configured to show all rows")


✅ Display settings configured to show all rows


In [2]:
from dataclasses import dataclass
from datetime import datetime, timedelta
from enum import Enum
import json
import os
from typing import Any, Optional, Tuple

import numpy as np
import pandas as pd

rng = np.random.default_rng(2025)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("✅ Libraries imported successfully")


✅ Libraries imported successfully


## Variables

In [3]:
# Realistic sizes based on real-world car rental industry data
n_users = 20_000         # User base across all countries
n_searches = 200_000     # Annual search volume - realistic for mid-size operation

_from_date = "2024-01-01"
_to_date = "2024-12-31"

hours = pd.date_range(_from_date, _to_date + " 23:00", freq="h")
sample_rows = 30_000     # Price observation sample size

n_comp = 10_000          # Competitor price observations  
comp_days = pd.date_range(_from_date, _to_date, freq="d")

# for simplicity lets keep the supplier locations inside USA, in cities like ``New York, Atlanta, Los Angeles, Chicago, Houston, Miami``
us_cities = ["New York", "Atlanta", "Los Angeles", "Chicago", "Houston", "Miami"]

# searches table (FK -> users, supplier_locations)
avg_searches_per_user_per_year: int = 80
start_date: str = pd.to_datetime(_from_date)
end_date: str = pd.to_datetime(_to_date)

# 70% of searches in final 2 weeks
search_period = 14

output_dir = "/Users/alejandro/workspace/car_rental_dynamic_book_prediction/data/sample"


## Suppliers

In [4]:
n_suppliers = 5
supplier_names = ["Avis", "Hertz", "Enterprise", "Sixt", "Budget"]
n_supplier_locations = 24
n_supplier_locations_per_city = 4

# Each city has 4 supplier locations (6 cities × 4 locations = 24 total supplier locations)
supplier_locations = pd.DataFrame({
    "location_id": range(1, n_supplier_locations + 1),
    "city": [city for city in us_cities for _ in range(n_supplier_locations_per_city)],
    "country": ["USA"] * n_supplier_locations
})

# suppliers table
suppliers = pd.DataFrame({
    "supplier_id": range(1, n_suppliers + 1),
    "supplier_name": supplier_names,
    "pricing_multiplier": [1.18, 1.20, 1.15, 1.10, 0.95]
})

# Supplier-Location mapping based on real market data
# Enterprise dominates (present everywhere), Hertz strong, Avis/Budget present, Sixt limited
# supplier_location_mapping = []

# Define realistic presence patterns based on market research
city_supplier_patterns = {
    "New York": {  # Major market - all suppliers present, 4 locations each
        "Enterprise": [1, 2, 3, 4],
        "Hertz": [1, 2, 3, 4], 
        "Avis": [1, 2, 3, 4],
        "Budget": [1, 2, 3],  # 3 locations
        "Sixt": [1]  # 1 location
    },
    "Los Angeles": {  # Major market - all suppliers present
        "Enterprise": [5, 6, 7, 8],
        "Hertz": [5, 6, 7, 8],
        "Avis": [5, 6, 7, 8], 
        "Budget": [5, 6, 7],  # 3 locations
        "Sixt": [5, 6]  # 2 locations
    },
    "Chicago": {  # Major market - all suppliers present
        "Enterprise": [9, 10, 11, 12],
        "Hertz": [9, 10, 11, 12],
        "Avis": [9, 10, 11],  # 3 locations
        "Budget": [9, 10, 11],  # 3 locations
        "Sixt": [9]  # 1 location
    },
    "Atlanta": {  # Hub city - strong presence
        "Enterprise": [13, 14, 15, 16],
        "Hertz": [13, 14, 15, 16],
        "Avis": [13, 14, 15],  # 3 locations
        "Budget": [13, 14],  # 2 locations
        "Sixt": [13]  # 1 location
    },
    "Houston": {  # Major market
        "Enterprise": [17, 18, 19, 20],
        "Hertz": [17, 18, 19],  # 3 locations
        "Avis": [17, 18, 19],  # 3 locations
        "Budget": [17, 18, 19],  # 3 locations
        "Sixt": [17]  # 1 location
    },
    "Miami": {  # Tourist destination - competitive market
        "Enterprise": [21, 22, 23, 24],
        "Hertz": [21, 22, 23, 24],  # Strong in tourist markets
        "Avis": [21, 22, 23, 24],  # Strong in tourist markets  
        "Budget": [21, 22, 23],  # 3 locations
        "Sixt": [21]  # 1 location only
    }
}

def supplier_location_mapping(suppliers: pd.DataFrame, city_supplier_patterns: dict) -> pd.DataFrame:
    supplier_id_map = dict(zip(suppliers['supplier_name'], suppliers['supplier_id']))
    
    # Pre-calculate total number of rows
    total_rows = sum(len(locations) 
                     for city_data in city_supplier_patterns.values() 
                     for locations in city_data.values())
    
    # Pre-allocate arrays
    supplier_ids = np.zeros(total_rows, dtype=int)
    location_ids = np.zeros(total_rows, dtype=int)
    supplier_names = []
    cities = []
    
    idx = 0
    for city, supplier_data in city_supplier_patterns.items():
        for supplier_name, locations in supplier_data.items():
            n_locations = len(locations)
            supplier_ids[idx:idx+n_locations] = supplier_id_map[supplier_name]
            location_ids[idx:idx+n_locations] = locations
            supplier_names.extend([supplier_name] * n_locations)
            cities.extend([city] * n_locations)
            idx += n_locations
    
    return pd.DataFrame({
        'supplier_id': supplier_ids,
        'location_id': location_ids,
        'supplier_name': supplier_names,
        'city': cities
    })

suppliers_df = supplier_location_mapping(suppliers, city_supplier_patterns)

In [5]:
# car classes
car_classes_df = pd.DataFrame({
    "car_class_id": range(1, 5),
    "car_class_name": ["economy", "compact", "suv", "luxury"],
    "probabilities": [0.32, 0.28, 0.25, 0.15]
})

## Location

In [6]:
# Map locations to cities
location_city_map = {
    1: "New York",
    2: "New York",
    3: "New York",
    4: "New York",
    5: "Los Angeles",
    6: "Los Angeles",
    7: "Los Angeles",
    8: "Los Angeles",
    9: "Chicago",
    10: "Chicago",
    11: "Chicago",
    12: "Chicago",
    13: "Atlanta",
    14: "Atlanta",
    15: "Atlanta",
    16: "Atlanta",
    17: "Houston",
    18: "Houston",
    19: "Houston",
    20: "Houston",
    21: "Miami",
    22: "Miami",
    23: "Miami",
    24: "Miami",
}

def get_location_weights(location_city_map):
    """
    Create location popularity weights based on research.
    Major airports and tourist destinations get higher weights.
    """
    
    # City weights based on research (high-demand cities)
    city_base_weights = {
        "New York": 1.5,  # Major tourist & business destination
        "Los Angeles": 1.5,  # Major tourist destination
        "Miami": 1.3,  # Major tourist destination
        "Chicago": 1.0,  # Business hub
        "Atlanta": 1.1,  # Major airport hub
        "Houston": 0.8,  # Lower tourism
    }

    # Airport vs local multiplier (38% airport, 46.8% local)
    # Assume location IDs 1, 5, 9, 13, 17, 21 are airports
    airport_locations = {1, 5, 9, 13, 17, 21}

    weights = {}
    for loc_id, city in location_city_map.items():
        base_weight = city_base_weights[city]
        if loc_id in airport_locations:
            weights[loc_id] = base_weight * 1.2  # Airport premium
        else:
            weights[loc_id] = base_weight

    # Normalize weights
    total = sum(weights.values())
    return {k: v / total for k, v in weights.items()}

location_weights = get_location_weights(location_city_map)

## User

In [7]:
# User locations table - countries where users can be located
user_countries = ["US", "CA", "CN", "EU", "CH", "JP", "MX", "SG", "AU"]
user_locations_df = pd.DataFrame({
    "user_location_id": range(1, len(user_countries) + 1),
    "country": user_countries
})

# users table (FK -> user_locations)
# Realistic distribution based on real-world car rental customer data in USA
user_location_probabilities = [
    0.78,   # US - Domestic users dominate (78%)
    0.08,   # CA - Largest international source, high car rental usage  
    0.04,   # MX - Large visitor volume but lower car rental penetration
    0.05,   # EU - High car rental penetration among European tourists
    0.015,  # CN - Growing but still limited car rental usage
    0.015,  # JP - Moderate tourism, some car rental usage
    0.01,   # CH - Small but affluent tourist segment
    0.005,  # SG - Limited volume but business travelers
    0.005    # AU - English-speaking, familiar with driving culture
]

class UserSegment(Enum):
    """User segments based on search and booking behavior."""

    NON_SEARCHER = "non_searcher"
    BROWSER_ONLY = "browser_only"
    SINGLE_TRIP = "single_trip"
    MULTI_TRIP = "multi_trip"
    FREQUENT_RENTER = "frequent_renter"


@dataclass
class SegmentConfig:
    """Configuration for each user segment."""

    segment: UserSegment
    population_pct: float
    searches_per_year_dist: str
    dist_params: dict[str, float]
    sessions_per_year_range: tuple[int, int]
    searches_per_session_range: tuple[int, int]
    planning_window_days: tuple[int, int]
    conversion_rate: float

def create_segment_configs() -> dict[UserSegment, SegmentConfig]:
    """Create configuration for each user segment based on research."""
    return {
        UserSegment.NON_SEARCHER: SegmentConfig(
            segment=UserSegment.NON_SEARCHER,
            population_pct=0.35,
            searches_per_year_dist="fixed",
            dist_params={"value": 0},
            sessions_per_year_range=(0, 0),
            searches_per_session_range=(0, 0),
            planning_window_days=(0, 0),
            conversion_rate=0.0,
        ),
        UserSegment.BROWSER_ONLY: SegmentConfig(
            segment=UserSegment.BROWSER_ONLY,
            population_pct=0.45,
            searches_per_year_dist="poisson",
            dist_params={"lambda": 5},
            sessions_per_year_range=(1, 3),
            searches_per_session_range=(2, 5),
            planning_window_days=(30, 90),
            conversion_rate=0.0,
        ),
        UserSegment.SINGLE_TRIP: SegmentConfig(
            segment=UserSegment.SINGLE_TRIP,
            population_pct=0.12,  # 60% of 20% active
            searches_per_year_dist="negative_binomial",
            dist_params={"r": 3, "mean": 25},
            sessions_per_year_range=(3, 8),
            searches_per_session_range=(4, 8),
            planning_window_days=(35, 49),  # 5-7 weeks
            conversion_rate=0.175,  # 17.5% average
        ),
        UserSegment.MULTI_TRIP: SegmentConfig(
            segment=UserSegment.MULTI_TRIP,
            population_pct=0.07,  # 35% of 20% active
            searches_per_year_dist="negative_binomial",
            dist_params={"r": 4, "mean": 50},
            sessions_per_year_range=(6, 16),
            searches_per_session_range=(5, 10),
            planning_window_days=(21, 35),  # 3-5 weeks
            conversion_rate=0.45,
        ),
        UserSegment.FREQUENT_RENTER: SegmentConfig(
            segment=UserSegment.FREQUENT_RENTER,
            population_pct=0.01,  # 5% of 20% active
            searches_per_year_dist="negative_binomial",
            dist_params={"r": 5, "mean": 90},
            sessions_per_year_range=(12, 30),
            searches_per_session_range=(4, 8),
            planning_window_days=(7, 21),  # 1-3 weeks
            conversion_rate=0.65,
        ),
    }

segment_configs = create_segment_configs()

In [8]:
def assign_user_segments(n_users: int, segment_configs: dict[UserSegment, SegmentConfig]) -> pd.DataFrame:
    """Assign users to segments based on population percentages."""
    users = []
    user_id = 1

    for segment, config in segment_configs.items():
        n_segment_users = int(n_users * config.population_pct)
        for _ in range(n_segment_users):
            users.append({"user_id": user_id, "segment": segment.value})
            user_id += 1

    # Handle rounding by adding remaining users to browser_only
    while user_id <= n_users:
        users.append(
            {"user_id": user_id, "segment": UserSegment.BROWSER_ONLY.value}
        )
        user_id += 1

    return pd.DataFrame(users)

users_df = assign_user_segments(n_users, segment_configs)
users_df["home_location_id"] = rng.choice(range(1, len(user_locations_df) + 1), size=n_users, p=user_location_probabilities)


## SEARCHES

In [9]:
# Hourly distribution for search sessions
        # Based on typical user behavior: peaks during work breaks and evenings
hour_weights = np.array([
    0.01,  # 0 - Midnight
    0.005, # 1
    0.005, # 2
    0.005, # 3
    0.005, # 4
    0.01,  # 5
    0.02,  # 6 - Early morning
    0.04,  # 7 - Morning commute
    0.06,  # 8 - Work start
    0.07,  # 9 - Morning peak
    0.075, # 10
    0.08,  # 11
    0.09,  # 12 - Lunch break peak
    0.085, # 13 - After lunch
    0.075, # 14
    0.07,  # 15
    0.065, # 16
    0.08,  # 17 - End of work day
    0.09,  # 18 - Evening peak
    0.095, # 19 - Prime evening
    0.09,  # 20 - Evening
    0.07,  # 21 - Late evening
    0.04,  # 22 - Night
    0.02   # 23 - Late night
])
hour_weights = hour_weights / hour_weights.sum()  # Normalize

In [10]:
def generate_searches_for_segment(config: SegmentConfig, n_users: int) -> list[int]:
    """Generate number of searches per user for a segment."""
    if config.searches_per_year_dist == "fixed":
        return [int(config.dist_params["value"])] * n_users

    elif config.searches_per_year_dist == "poisson":
        return rng.poisson(config.dist_params["lambda"], n_users).tolist()

    elif config.searches_per_year_dist == "negative_binomial":      
        r = config.dist_params["r"]
        mean = config.dist_params["mean"]
        p = r / (r + mean)
        return rng.negative_binomial(r, p, n_users).tolist()

    return [0] * n_users

In [11]:
def generate_trip_dates(start_date: datetime, end_date: datetime, n_trips: int) -> list[datetime]:
    """Generate trip dates for a user based on segment."""
    if n_trips == 0:
        return []

    # Spread trips throughout the year
    days_range = (end_date - start_date).days
    trip_days = sorted(rng.integers(0, days_range, n_trips))

    return [start_date + timedelta(days=int(day)) for day in trip_days]

In [12]:
def distribute_searches_to_sessions(
        n_searches: int, n_sessions: int, config: SegmentConfig
    ) -> list[int]:
    """Distribute total searches across sessions."""
    if n_sessions == 0:
        return []

    min_per_session, max_per_session = config.searches_per_session_range

    # Start with minimum searches per session
    searches_per_session = [min_per_session] * n_sessions
    remaining = n_searches - sum(searches_per_session)

    # Distribute remaining searches
    session_indices = list(range(n_sessions))
    while remaining > 0 and session_indices:
        idx = rng.choice(session_indices)
        if searches_per_session[idx] < max_per_session:
            searches_per_session[idx] += 1
            remaining -= 1
        else:
            session_indices.remove(idx)

    return searches_per_session

In [13]:
def create_session_searches(
        user_id: int, 
        session_start: datetime, 
        n_searches: int, 
        location_weights: dict[int, float], 
        car_classes_df: pd.DataFrame
    ) -> list[dict[str, Any]]:
    """Create individual searches within a session."""
    searches = []

    # Session duration: 15-45 minutes
    session_duration_minutes = rng.integers(15, 46)

    for _ in range(n_searches):
        # Spread searches throughout the session
        minute_offset = rng.integers(0, session_duration_minutes)
        search_time = session_start + timedelta(minutes=int(minute_offset))

        # Select location and car class
        location_id = rng.choice(
            list(location_weights.keys()),
            p=list(location_weights.values()),
        )
        car_class = rng.choice(
            list(car_classes_df["car_class_name"].to_list()),
            p=list(car_classes_df["probabilities"].to_list()),
        )

        searches.append(
            {
                "user_id": user_id,
                "search_ts": search_time,
                "location_id": location_id,
                "car_class": car_class,
                "session_id": f"{user_id}_{session_start.strftime('%Y%m%d%H%M')}",
            }
        )

    return searches

In [14]:
def create_session_near_trip(
        user_id: int, config: SegmentConfig, n_searches: int, trip_date: datetime, search_period: int
    ) -> list[dict[str, Any]]:
        """Create a search session clustered near a trip date."""
        min_days, max_days = config.planning_window_days
        days_before = rng.integers(min_days, max_days + 1)

        # 70% of searches in final 2 weeks
        if days_before > search_period:
            # Early planning session
            session_date = trip_date - timedelta(days=int(days_before))
            session_searches = max(1, int(n_searches * 0.3))
        else:
            # Intensive search period
            session_date = trip_date - timedelta(days=int(rng.integers(1, 15)))
            session_searches = n_searches

        # Add realistic hour to session date
        hour = rng.choice(24, p=hour_weights)
        minute = rng.integers(0, 60)
        session_datetime = session_date.replace(hour=int(hour), minute=int(minute))

        return create_session_searches(user_id, session_datetime, session_searches, location_weights, car_classes_df)

In [15]:
def create_random_session(
        user_id: int, 
        start_date, 
        end_date, 
        n_searches: int, 
        location_weights: dict[int, float], 
        car_classes_df: pd.DataFrame
    ) -> list[dict[str, Any]]:
        """Create a random search session (for browsers)."""
        days_range = (end_date - start_date).days
        session_start = start_date + timedelta(
            days=int(rng.integers(0, days_range))
        )
        # Add realistic hour to session date  
        hour = rng.choice(24, p=hour_weights)
        minute = rng.integers(0, 60)
        session_datetime = session_start.replace(hour=int(hour), minute=int(minute))
        
        return create_session_searches(user_id, session_datetime, n_searches, location_weights, car_classes_df)

In [16]:
def generate_search_sessions(
        user_id: int,
        config: SegmentConfig,
        n_searches: int,
        trip_dates: list[datetime],
        search_period: int,
        start_date: datetime,
        end_date: datetime,
        location_weights: dict[int, float],
        car_classes_df: pd.DataFrame

    ) -> list[dict[str, Any]]:
    """Generate search sessions for a user."""
    if n_searches == 0:
        return []

    sessions = []

    # Determine number of sessions
    min_sessions, max_sessions = config.sessions_per_year_range
    n_sessions = min(
        max(
            rng.integers(min_sessions, max_sessions + 1),
            1 if n_searches > 0 else 0,
        ),
        n_searches,
    )

    if n_sessions == 0:
        return []

    # Distribute searches across sessions
    searches_per_session = distribute_searches_to_sessions(
        n_searches, n_sessions, config
    )

    # Generate sessions
    if trip_dates:
        # Cluster sessions around trip dates
        for trip_date, session_searches in zip(
            trip_dates[:n_sessions], searches_per_session
        ):
            if session_searches > 0:
                session = create_session_near_trip(
                    user_id, config, session_searches, trip_date, search_period
                )
                sessions.extend(session)
    else:
        # Random distribution throughout the year for browsers
        for session_searches in searches_per_session:
            if session_searches > 0:
                session = create_random_session(
                    user_id, start_date, end_date, session_searches, location_weights, car_classes_df
                )
                sessions.extend(session)

    return sessions

In [17]:
def generate_searches(users_df: pd.DataFrame, 
                      segment_configs: dict[UserSegment, SegmentConfig],
                      search_period: int,
                      start_date: datetime,
                      end_date: datetime,
                      location_weights: dict[int, float],
                      car_classes_df: pd.DataFrame
                      ) -> pd.DataFrame:
    """
    Generate realistic search data with proper user segmentation.

    Returns:
        pd.DataFrame: Search data with columns:
            - search_id: Unique search identifier
            - user_id: User identifier
            - location_id: Rental location
            - search_ts: Search timestamp
            - car_class: Type of car
            - user_segment: User's segment
            - session_id: Session identifier
    """
    all_searches = []
    search_id = 1

    # Generate searches for each segment
    for segment, config in segment_configs.items():
        segment_users = users_df[users_df["segment"] == segment.value]
        n_segment_users = len(segment_users)

        if n_segment_users == 0:
            continue

        # Generate searches per user
        searches_per_user = generate_searches_for_segment(
            config, n_segment_users
        )

        # Process each user
        for idx, (_, user_row) in enumerate(segment_users.iterrows()):
            user_id = user_row["user_id"]
            n_searches = searches_per_user[idx]

            if n_searches == 0:
                continue

            # Generate trip dates for active searchers
            n_trips = 0
            if segment == UserSegment.SINGLE_TRIP:
                n_trips = 1
            elif segment == UserSegment.MULTI_TRIP:
                n_trips = rng.integers(2, 4)
            elif segment == UserSegment.FREQUENT_RENTER:
                n_trips = rng.integers(4, 7)

            trip_dates = generate_trip_dates(start_date, end_date, n_trips)

            # Generate search sessions
            user_searches = generate_search_sessions(
                user_id, 
                config, 
                n_searches, 
                trip_dates, 
                search_period, 
                start_date, 
                end_date, 
                location_weights, 
                car_classes_df
            )

            # Add segment info and search IDs
            for search in user_searches:
                search["search_id"] = search_id
                search["user_segment"] = segment.value
                all_searches.append(search)
                search_id += 1

    # Convert to DataFrame and sort by timestamp
    searches_df = pd.DataFrame(all_searches)
    if not searches_df.empty:
        searches_df = searches_df.sort_values("search_ts").reset_index(drop=True)
        searches_df["search_id"] = range(1, len(searches_df) + 1)

    return searches_df

In [18]:
searches_df = generate_searches(users_df, 
                                segment_configs, 
                                search_period, 
                                start_date, 
                                end_date,
                                location_weights,
                                car_classes_df)

In [19]:
searches_df.head(5)
searches_df[["location_id", "car_class", "search_ts"]].describe()

,user_id,search_ts,location_id,car_class,session_id,search_id,user_segment
0,16067,2023-11-14 14:54:00,14,compact,16067_202311141440,1,single_trip
1,16064,2023-11-14 18:42:00,8,suv,16064_202311141822,2,single_trip
2,17978,2023-11-15 19:15:00,5,compact,17978_202311151853,3,single_trip
3,16053,2023-11-16 13:24:00,21,compact,16053_202311161253,4,single_trip
4,16053,2023-11-16 13:25:00,15,economy,16053_202311161253,5,single_trip


,location_id,search_ts
count,57282.000000,57282
mean,11.571104,2024-06-27 22:22:31.552320
min,1.000000,2023-11-14 14:54:00
25%,5.000000,2024-03-28 20:52:45
50%,11.000000,2024-06-28 17:16:30
75%,18.000000,2024-09-27 04:37:15
max,24.000000,2024-12-31 00:02:00
std,7.225129,NaN


In [20]:
def generate_statistics_report(searches_df, n_users, location_city_map):
    """
    Generate a statistics report to validate the generated data.
    """
    print("=== Car Rental Search Data Statistics ===\n")

    # Overall statistics
    print(f"Total searches: {len(searches_df):,}")
    print(f"Total users: {searches_df['user_id'].nunique():,}")
    print(
        f"Average searches per user: {len(searches_df) / searches_df['user_id'].nunique():.2f}"
    )

    # User segment distribution
    print("\nUser Segment Distribution:")
    segment_counts = searches_df.groupby("user_segment")["user_id"].nunique()
    for segment, count in segment_counts.items():
        pct = count / n_users * 100
        print(f"  {segment}: {count:,} users ({pct:.1f}%)")

    # Searches per user distribution
    searches_per_user = searches_df.groupby("user_id").size()
    print(f"\nSearches per User Distribution:")
    print(f"  Min: {searches_per_user.min()}")
    print(f"  25th percentile: {searches_per_user.quantile(0.25):.0f}")
    print(f"  Median: {searches_per_user.median():.0f}")
    print(f"  75th percentile: {searches_per_user.quantile(0.75):.0f}")
    print(f"  Max: {searches_per_user.max()}")
    print(
        f"  Users with 0 searches: {n_users - searches_df['user_id'].nunique():,}"
    )

    # Location distribution
    print("\nTop 10 Most Searched Locations:")
    location_dist = searches_df["location_id"].value_counts()
    for loc_id, count in location_dist.head(10).items():
        pct = count / len(searches_df) * 100
        # city_map = {1: "NY", 5: "LA", 9: "CHI", 13: "ATL", 17: "HOU", 21: "MIA"}
        city = location_city_map.get(loc_id, "Unknown")
        print(f"  Location {loc_id:2d} ({city}): {count:,} searches ({pct:.1f}%)")

    # Car class distribution
    print("\nCar Class Distribution:")
    car_class_dist = searches_df["car_class"].value_counts()
    for car_class, count in car_class_dist.items():
        pct = count / len(searches_df) * 100
        print(f"  {car_class}: {count:,} searches ({pct:.1f}%)")

    # Temporal patterns
    print("\nTemporal Patterns:")
    searches_df["hour"] = searches_df["search_ts"].dt.hour
    searches_df["weekday"] = searches_df["search_ts"].dt.day_name()
    searches_df["month"] = searches_df["search_ts"].dt.month
    searches_df["month_name"] = searches_df["search_ts"].dt.month_name()

    print("  Searches by Month:")
    month_order = [
        "January",
        "February",
        "March",
        "April",
        "May",
        "June",
        "July",
        "August",
        "September",
        "October",
        "November",
        "December",
    ]
    for month in month_order:
        if month in searches_df["month_name"].values:
            count = (searches_df["month_name"] == month).sum()
            pct = count / len(searches_df) * 100
            print(f"    {month}: {count:,} searches ({pct:.1f}%)")

    print("\n  Searches by Day of Week:")
    day_order = [
        "Monday",
        "Tuesday",
        "Wednesday",
        "Thursday",
        "Friday",
        "Saturday",
        "Sunday",
    ]
    for day in day_order:
        if day in searches_df["weekday"].values:
            count = (searches_df["weekday"] == day).sum()
            pct = count / len(searches_df) * 100
            print(f"    {day}: {count:,} searches ({pct:.1f}%)")


generate_statistics_report(searches_df, n_users, location_city_map)

=== Car Rental Search Data Statistics ===

Total searches: 57,282
Total users: 12,929
Average searches per user: 4.43

User Segment Distribution:
  browser_only: 8,929 users (44.6%)
  frequent_renter: 200 users (1.0%)
  multi_trip: 1,400 users (7.0%)
  single_trip: 2,400 users (12.0%)

Searches per User Distribution:
  Min: 1
  25th percentile: 2
  Median: 4
  75th percentile: 6
  Max: 41
  Users with 0 searches: 7,071

Top 10 Most Searched Locations:
  Location  1 (New York): 3,537 searches (6.2%)
  Location  5 (Los Angeles): 3,445 searches (6.0%)
  Location 21 (Miami): 3,058 searches (5.3%)
  Location  8 (Los Angeles): 2,884 searches (5.0%)
  Location  3 (New York): 2,882 searches (5.0%)
  Location  6 (Los Angeles): 2,800 searches (4.9%)
  Location  7 (Los Angeles): 2,796 searches (4.9%)
  Location  2 (New York): 2,782 searches (4.9%)
  Location  4 (New York): 2,767 searches (4.8%)
  Location 13 (Atlanta): 2,598 searches (4.5%)

Car Class Distribution:
  economy: 18,254 searches (31.

## Rental & Competitor Prices

In [21]:
price_sensitivity = {
    "economy": 1.5,    # Most price sensitive
    "compact": 1.2,
    "suv": 0.8,
    "luxury": 0.5,     # Least price sensitive
}

# Expected price ranges by car class (for conversion calculations)
expected_prices = {
    "economy": {"min": 35, "typical": 50, "max": 80},
    "compact": {"min": 40, "typical": 58, "max": 95},
    "suv": {"min": 60, "typical": 85, "max": 140},
    "luxury": {"min": 100, "typical": 140, "max": 250},
}

price_sensitivity = {
    "economy": 1.5,    # Most price sensitive
    "compact": 1.2,
    "suv": 0.8,
    "luxury": 0.5,     # Least price sensitive
}

monthly_multipliers = {
    1: 0.80,   # January - lowest
    2: 0.85,   # February - low
    3: 0.95,   # March - spring break begins
    4: 1.00,   # April - moderate
    5: 1.15,   # May - pre-summer high
    6: 1.25,   # June - summer peak begins
    7: 1.30,   # July - peak summer
    8: 1.25,   # August - still peak
    9: 1.05,   # September - post-summer
    10: 0.95,  # October - moderate
    11: 0.90,  # November - low season
    12: 1.10,  # December - holiday travel
}

# Day of week multipliers
dow_multipliers = {
    0: 1.10,  # Monday - business travel
    1: 1.15,  # Tuesday - peak business
    2: 1.15,  # Wednesday - peak business
    3: 1.12,  # Thursday - business + early weekend
    4: 1.05,  # Friday - mixed
    5: 0.95,  # Saturday - leisure discount
    6: 0.95,  # Sunday - leisure discount
}

# Location premiums (airport locations)
airport_locations = {1, 5, 9, 13, 17, 21}  # Major airports
airport_premium = 1.22  # 22% premium for airports


def define_holidays() -> dict[Tuple[int, int], float]:
    """Define major holidays and their price multipliers."""
    return {
        (1, 1): 1.5,    # New Year's Day
        (2, 14): 1.3,   # Valentine's Day
        (3, 17): 1.4,   # St. Patrick's Day (varies)
        (5, 25): 1.4,   # Memorial Day weekend (last Monday)
        (7, 4): 1.5,    # July 4th
        (9, 1): 1.4,    # Labor Day (first Monday)
        (11, 22): 1.6,  # Thanksgiving (4th Thursday)
        (12, 25): 1.8,  # Christmas
        (12, 31): 1.7,  # New Year's Eve
    }

holidays = define_holidays()


In [22]:
def get_booking_lead_time_distribution() -> np.ndarray:
    """Get probability distribution for booking lead times."""
    # Based on research: most bookings happen 1-30 days out
    # Create a distribution that favors 7-21 days
    days = np.arange(1, 61)
    
    # Peak around 14 days, decay after
    weights = np.exp(-0.5 * ((days - 14) / 10) ** 2)
    
    # Boost last-minute bookings slightly
    weights[:7] *= 1.5
    
    # Normalize
    return weights / weights.sum()

In [23]:
def get_holiday_multiplier(date: pd.Timestamp, holidays: dict[tuple[int, int], float]) -> float:
    """Get holiday price multiplier for a given date."""
    # Check exact date
    date_key = (date.month, date.day)
    if date_key in holidays:
        return holidays[date_key]
    
    # Check if near a holiday (within 3 days)
    for holiday_date, multiplier in holidays.items():
        holiday = pd.Timestamp(date.year, holiday_date[0], holiday_date[1])
        if abs((date - holiday).days) <= 3:
            # Gradual increase/decrease around holidays
            distance = abs((date - holiday).days)
            return 1.0 + (multiplier - 1.0) * (1 - distance / 4)
    
    return 1.0

In [24]:
def calculate_price(
        base_prices: dict[str, float],
        supplier_multipliers: pd.DataFrame,
        monthly_multipliers: dict[int, float],
        dow_multipliers: dict[int, float],
        airport_locations: set[int],
        airport_premium: float,
        holidays: dict[Tuple[int, int], float],
        pickup_date: pd.Timestamp,
        location_id: int,
        supplier_id: int,
        car_class: str,
        obs_time: pd.Timestamp,
        days_until_pickup: float,
    ) -> float:
    """Calculate price based on all factors."""
    # Start with base price
    base = base_prices[car_class]["typical"]
    
    # Apply supplier multiplier
    base *= supplier_multipliers[supplier_multipliers["supplier_id"] == supplier_id]["pricing_multiplier"].values[0]
    
    # Apply monthly seasonality
    base *= monthly_multipliers[pickup_date.month]
    
    # Apply day of week pattern
    base *= dow_multipliers[pickup_date.weekday()]
    
    # Apply location premium
    if location_id in airport_locations:
        base *= airport_premium
    
    # Apply holiday premium
    holiday_mult = get_holiday_multiplier(pickup_date, holidays)
    base *= holiday_mult
    
    # Apply booking lead time adjustment
    # Last-minute bookings (< 7 days) are more expensive
    if days_until_pickup < 7:
        base *= 1.15 + (7 - days_until_pickup) * 0.03
    elif days_until_pickup > 45:
        # Far advance bookings get small discount
        base *= 0.95
    
    # Apply supply/demand variation (random ±15%)
    base *= rng.uniform(0.85, 1.15)
    
    # Apply time of day variation for obs_time
    hour = obs_time.hour
    if 6 <= hour <= 9 or 17 <= hour <= 20:  # Peak hours
        base *= 1.05
    elif 0 <= hour <= 5:  # Night hours
        base *= 0.98
    
    return max(25.0, base)  # Minimum price floor

In [25]:
def generate_rental_prices(
        start_date: pd.Timestamp,
        end_date: pd.Timestamp,
        locations: list[int],
        suppliers: list[int],
        car_classes: list[str],
        base_prices: dict[str, float],
        monthly_multipliers: dict[int, float],
        supplier_multipliers: pd.DataFrame,
        dow_multipliers: dict[int, float],
        airport_locations: set[int],
        airport_premium: float,
        holidays: dict[tuple[int, int], float],
    ) -> pd.DataFrame:
    """
    Generate hourly rental price observations.
    
    Args:
        n_records: Number of price records to generate
        start_date: Start date for observations
        end_date: End date for observations
        locations: List of valid location IDs
        suppliers: List of valid supplier IDs
        car_classes: List of valid car classes
        
    Returns:
        DataFrame with rental price observations
    """

    # Generate observation timestamps (hourly)
    date_range = pd.date_range(start_date, end_date, freq='h')
    obs_timestamps = rng.choice(date_range, size=date_range.shape[0])
    
    # Generate pickup dates (future dates from observation)
    # Most observations are for pickups 1-60 days in the future
    days_ahead = rng.choice(
        np.arange(1, 61),
        size=date_range.shape[0],
        p=get_booking_lead_time_distribution()
    )
    
    prices = []
    for i in range(date_range.shape[0]):
        obs_ts = pd.Timestamp(obs_timestamps[i])
        pickup_date = obs_ts + timedelta(days=int(days_ahead[i]))
        
        # Select attributes
        location_id = rng.choice(locations)
        supplier_id = rng.choice(suppliers)
        car_class = rng.choice(car_classes)
        
        # Calculate price
        price = calculate_price(
            base_prices,
            supplier_multipliers,
            monthly_multipliers,
            dow_multipliers,
            airport_locations,
            airport_premium,
            holidays,
            pickup_date,
            location_id,
            supplier_id,
            car_class,
            obs_ts,
            days_ahead[i]
        )
        
        # Calculate availability (inverse relationship with price)
        # Higher prices = lower availability
        base_availability = 15
        price_factor = price / base_prices[car_class]["typical"]
        availability = max(0, int(base_availability / price_factor + rng.normal(0, 3)))
        
        prices.append({
            "price_id": i + 1,
            "location_id": location_id,
            "supplier_id": supplier_id,
            "car_class": car_class,
            "pickup_date": pickup_date,
            "obs_ts": obs_ts,
            "current_price": round(price, 2),
            "available_cars": availability,
            "days_until_pickup": int(days_ahead[i]),
        })
    
    df = pd.DataFrame(prices)
    
    # Sort by observation timestamp
    df = df.sort_values("obs_ts").reset_index(drop=True)
    df["price_id"] = range(1, len(df) + 1)
    
    return df

In [26]:
def generate_competitor_prices(
        start_date: pd.Timestamp,
        end_date: pd.Timestamp,
        locations: list[int],
        car_classes: list[str],
        base_prices: dict[str, float],
        monthly_multipliers: dict[int, float],
        supplier_multipliers: pd.DataFrame,
        dow_multipliers: dict[int, float],
        airport_locations: set[int],
        airport_premium: float,
        holidays: dict[tuple[int, int], float],
    ) -> pd.DataFrame:
    """
    Generate daily competitor price observations (minimum across all suppliers).
    
    Args:
        n_records: Number of price records to generate
        start_date: Start date for observations
        end_date: End date for observations
        locations: List of valid location IDs
        car_classes: List of valid car classes
        
    Returns:
        DataFrame with competitor price observations
    """
    # Generate observation dates (daily)
    date_range = pd.date_range(start_date, end_date, freq='D')
    obs_dates = rng.choice(date_range, size=date_range.shape[0])
    
    # Generate pickup dates
    days_ahead = rng.choice(
        np.arange(1, 61),
        size=date_range.shape[0],
        p=get_booking_lead_time_distribution()
    )
    
    comp_prices = []
    for i in range(date_range.shape[0]):
        obs_date = pd.Timestamp(obs_dates[i])
        pickup_date = obs_date + timedelta(days=int(days_ahead[i]))
        
        # Select attributes
        location_id = rng.choice(locations)
        car_class = rng.choice(car_classes)
        
        # Calculate minimum price across all suppliers
        supplier_prices = []
        for supplier_id in range(1, 6):  # All 5 suppliers
            price = calculate_price(
                base_prices,
                supplier_multipliers,
                monthly_multipliers,
                dow_multipliers,
                airport_locations,
                airport_premium,
                holidays,
                pickup_date,
                location_id,
                supplier_id,
                car_class,
                obs_date,
                days_ahead[i]
            )
            supplier_prices.append(price)
        
        # Competitor price is minimum with some noise
        min_price = min(supplier_prices) * rng.uniform(0.95, 1.02)
        
        comp_prices.append({
            "comp_id": i + 1,
            "location_id": location_id,
            "car_class": car_class,
            "pickup_date": pickup_date,
            "obs_date": obs_date,
            "comp_min_price": round(min_price, 2),
            "days_until_pickup": int(days_ahead[i]),
        })
    
    df = pd.DataFrame(comp_prices)
    
    # Sort by observation date
    df = df.sort_values("obs_date").reset_index(drop=True)
    df["comp_id"] = range(1, len(df) + 1)
    
    return df

In [27]:
print("Generating rental prices...")
rental_prices_df = generate_rental_prices(
    start_date=start_date,
    end_date=end_date,
    locations=supplier_locations["location_id"].to_list(),
    suppliers=suppliers["supplier_id"].to_list(),
    car_classes=car_classes_df["car_class_name"].to_list(),
    base_prices=expected_prices,
    monthly_multipliers=monthly_multipliers,
    supplier_multipliers=suppliers,
    dow_multipliers=dow_multipliers,
    airport_locations=airport_locations,
    airport_premium=airport_premium,
    holidays=holidays,
)

# Generate competitor prices
print("\nGenerating competitor prices...")
competitor_prices_df = generate_competitor_prices(
    start_date=start_date,
    end_date=end_date,
    locations=supplier_locations["location_id"].to_list(),
    car_classes=car_classes_df["car_class_name"].to_list(),
    base_prices=expected_prices,
    monthly_multipliers=monthly_multipliers,
    supplier_multipliers=suppliers,
    dow_multipliers=dow_multipliers,
    airport_locations=airport_locations,
    airport_premium=airport_premium,
    holidays=holidays,
)

Generating rental prices...

Generating competitor prices...


In [28]:
def generate_price_summary(prices_df: pd.DataFrame, airport_locations: set[int], suppliers: pd.DataFrame) -> None:
    """Print summary statistics for generated prices."""
    print("\n" + "="*60)
    print("PRICE GENERATION SUMMARY")
    print("="*60)
    
    print(f"\nTotal price records: {len(prices_df):,}")
    
    # Price statistics by car class
    print("\nAverage prices by car class:")
    for car_class in ["economy", "compact", "suv", "luxury"]:
        if car_class in prices_df["car_class"].values:
            avg_price = prices_df[prices_df["car_class"] == car_class]["current_price"].mean()
            print(f"  {car_class}: ${avg_price:.2f}")
    
    # Price statistics by supplier
    print("\nAverage prices by supplier:")
    for supplier_id in sorted(prices_df["supplier_id"].unique()):
        avg_price = prices_df[prices_df["supplier_id"] == supplier_id]["current_price"].mean()
        supplier_name = suppliers[suppliers["supplier_id"] == supplier_id]["supplier_name"].values[0]
        print(f"  {supplier_name}: ${avg_price:.2f}")
    
    # Monthly patterns
    if "pickup_date" in prices_df.columns:
        prices_df["pickup_month"] = pd.to_datetime(prices_df["pickup_date"]).dt.month
        print("\nAverage prices by month:")
        monthly_avg = prices_df.groupby("pickup_month")["current_price"].mean().sort_index()
        for month, price in monthly_avg.items():
            month_name = pd.Timestamp(2024, int(month), 1).strftime("%B")
            print(f"  {month_name}: ${price:.2f}")
    
    # Location patterns
    print("\nAirport vs Downtown prices:")
    airport_prices = prices_df[prices_df["location_id"].isin(airport_locations)]["current_price"].mean()
    downtown_prices = prices_df[~prices_df["location_id"].isin(airport_locations)]["current_price"].mean()
    print(f"  Airport locations: ${airport_prices:.2f}")
    print(f"  Downtown locations: ${downtown_prices:.2f}")
    print(f"  Airport premium: {(airport_prices/downtown_prices - 1)*100:.1f}%")

In [29]:
generate_price_summary(rental_prices_df, airport_locations, suppliers)


PRICE GENERATION SUMMARY

Total price records: 8,761

Average prices by car class:
  economy: $73.48
  compact: $84.57
  suv: $125.03
  luxury: $204.67

Average prices by supplier:
  Avis: $128.15
  Hertz: $131.06
  Enterprise: $126.89
  Sixt: $120.14
  Budget: $102.18

Average prices by month:
  January: $92.89
  February: $99.02
  March: $109.96
  April: $113.35
  May: $133.79
  June: $135.59
  July: $158.93
  August: $139.58
  September: $119.20
  October: $105.02
  November: $108.16
  December: $141.82

Airport vs Downtown prices:
  Airport locations: $138.76
  Downtown locations: $115.79
  Airport premium: 19.8%


## Bookings

In [30]:
# Realistic conversion rate: 2.5% (200,000 searches → 5,000 bookings)
n_bookings = int(searches_df.shape[0] * 0.025)

# segment_configs
segment_conversion_rates = {
    user_segment.value: config.conversion_rate
    for user_segment, config in segment_configs.items()
}

In [31]:
def validate_inputs(searches_df: pd.DataFrame, rental_prices_df: pd.DataFrame) -> None:
    """Validate required columns exist."""
    search_required = ["search_id", "user_id", "location_id", "car_class", "search_ts"]
    price_required = ["location_id", "supplier_id", "car_class", "pickup_date", "obs_ts", "current_price"]
    
    missing_search = set(search_required) - set(searches_df.columns)
    missing_price = set(price_required) - set(rental_prices_df.columns)
    
    if missing_search:
        raise ValueError(f"Missing columns in searches_df: {missing_search}")
    if missing_price:
        raise ValueError(f"Missing columns in rental_prices_df: {missing_price}")


In [32]:
def get_competitor_price(
        search: pd.Series, competitor_prices_df: pd.DataFrame
    ) -> Optional[float]:
    """Get competitor price for comparison."""
    search_date = pd.Timestamp(search["search_ts"].date())
    
    # Ensure obs_date is datetime
    competitor_prices_df["obs_date"] = pd.to_datetime(competitor_prices_df["obs_date"])
    
    mask = (
        (competitor_prices_df["obs_date"] <= search_date) &
        (competitor_prices_df["location_id"] == search["location_id"]) &
        (competitor_prices_df["car_class"] == search["car_class"])
    )
    
    matches = competitor_prices_df[mask]
    if matches.empty:
        return None
    
    # Get most recent competitor price
    return matches.sort_values("obs_date").iloc[-1]["comp_min_price"]

In [33]:
def find_available_prices(
        search: pd.Series,
        rental_prices_df: pd.DataFrame,
        competitor_prices_df: Optional[pd.DataFrame] = None,
    ) -> pd.DataFrame:
    """Find rental prices available at the time of search."""
    # Prices must be:
    # 1. Observed before or at search time
    # 2. For pickup dates after search time
    # 3. Matching location and car class
    
    search_time = search["search_ts"]
    
    # Filter rental prices
    mask = (
        (rental_prices_df["obs_ts"] <= search_time) &
        (rental_prices_df["pickup_date"] > search_time) &
        (rental_prices_df["location_id"] == search["location_id"]) &
        (rental_prices_df["car_class"] == search["car_class"]) &
        (rental_prices_df["available_cars"] > 0)  # Must have availability
    )
    
    available = rental_prices_df[mask].copy()
    
    if available.empty:
        return available
    
    # For each supplier/pickup_date combo, get the most recent observation
    available = available.sort_values("obs_ts").groupby(
        ["supplier_id", "pickup_date"]
    ).last().reset_index()
    
    # Add competitor price if available
    if competitor_prices_df is not None:
        comp_price = get_competitor_price(search, competitor_prices_df)
        if comp_price is not None:
            available["competitor_price"] = comp_price
        else:
            available["competitor_price"] = available["current_price"].min()
    
    # Calculate days until pickup
    available["days_until_pickup"] = (
        available["pickup_date"] - search_time
    ).dt.total_seconds() / 86400
    
    # Filter to reasonable booking window (1-60 days)
    available = available[
        (available["days_until_pickup"] >= 1) & 
        (available["days_until_pickup"] <= 60)
    ]
    
    return available

In [34]:
def generate_booking_delay() -> float:
    """Generate realistic delay between search and booking."""
    rand = rng.random()
    
    if rand < 0.4:
        # Within 1 hour
        return rng.exponential(0.3)
    elif rand < 0.7:
        # 1-24 hours
        return rng.uniform(1, 24)
    elif rand < 0.9:
        # 1-7 days
        return rng.uniform(24, 168)
    else:
        # 7+ days
        return 168 + rng.exponential(120)

In [35]:
def attempt_booking(
        search: pd.Series, 
        available_prices: pd.DataFrame,
        segment_conversion_rates: dict[str, float],
        expected_prices: dict[str, dict[str, float]],
        price_sensitivity: dict[str, float],
    ) -> Optional[dict]:
    """Decide if search converts to booking based on prices."""
    # Get base conversion rate
    segment = search.get("user_segment", "single_trip")
    base_rate = segment_conversion_rates.get(segment, 0.025)
    
    if base_rate == 0:
        return None
    
    # Find best price option
    best_option = available_prices.loc[available_prices["current_price"].idxmin()]
    price = best_option["current_price"]
    
    # Calculate price-adjusted conversion rate
    car_class = search["car_class"]
    expected = expected_prices[car_class]["typical"]
    price_ratio = price / expected
    
    # Adjust conversion rate based on price
    # If price is at expected level, no adjustment
    # If price is 50% higher, reduce conversion by sensitivity factor
    sensitivity = price_sensitivity[car_class]
    price_adjustment = 1.0 - sensitivity * max(0, price_ratio - 1.0)
    price_adjustment = max(0.1, min(2.0, price_adjustment))  # Bounds
    
    final_rate = base_rate * price_adjustment
    
    # Random decision
    if rng.random() >= final_rate:
        return None
    
    # Create booking
    booking_delay_hours = generate_booking_delay()
    booking_ts = search["search_ts"] + timedelta(hours=booking_delay_hours)
    
    # Ensure booking happens before pickup
    max_booking_time = best_option["pickup_date"] - timedelta(hours=4)
    if booking_ts > max_booking_time:
        booking_ts = max_booking_time
        booking_delay_hours = (booking_ts - search["search_ts"]).total_seconds() / 3600
    
    return {
        "search_id": search["search_id"],
        "user_id": search["user_id"],
        "supplier_id": best_option["supplier_id"],
        "location_id": search["location_id"],
        "car_class": search["car_class"],
        "pickup_date": best_option["pickup_date"],
        "search_ts": search["search_ts"],
        "booking_ts": booking_ts,
        "booking_delay_hours": booking_delay_hours,
        "search_price": price,  # Price at search time
        "booked_price": price,  # Could be different in reality
        "price_rank": 1,  # Best price = 1
        "competitor_price": best_option.get("competitor_price", np.nan),
        "days_until_pickup": best_option["days_until_pickup"],
    }

In [36]:
def finalize_bookings(
        bookings_df: pd.DataFrame, searches_df: pd.DataFrame
    ) -> pd.DataFrame:
    """Finalize booking data with proper IDs and sorting."""
    # Sort by booking timestamp
    bookings_df = bookings_df.sort_values("booking_ts").reset_index(drop=True)
    
    # Add booking IDs
    bookings_df["booking_id"] = range(1, len(bookings_df) + 1)
    
    # Add user segment if available
    if "user_segment" in searches_df.columns:
        segment_map = searches_df.set_index("search_id")["user_segment"].to_dict()
        bookings_df["user_segment"] = bookings_df["search_id"].map(segment_map)
    
    # Reorder columns
    column_order = [
        "booking_id", "search_id", "user_id", "supplier_id", "location_id",
        "car_class", "pickup_date", "search_ts", "booking_ts", 
        "booking_delay_hours", "days_until_pickup", "search_price", 
        "booked_price", "competitor_price", "price_rank"
    ]
    
    # Add user_segment if present
    if "user_segment" in bookings_df.columns:
        column_order.append("user_segment")
    
    return bookings_df[column_order]

In [37]:
def print_booking_summary(bookings_df: pd.DataFrame) -> None:
    """Print comprehensive booking summary."""
    print("\n" + "="*60)
    print("PRICE-AWARE BOOKING SUMMARY")
    print("="*60)
    
    print(f"\nTotal bookings: {len(bookings_df):,}")
    
    # Price statistics
    print("\nPrice Statistics:")
    print(f"  Average booked price: ${bookings_df['booked_price'].mean():.2f}")
    print(f"  Price range: ${bookings_df['booked_price'].min():.2f} - ${bookings_df['booked_price'].max():.2f}")
    
    # Price by car class
    print("\nAverage price by car class:")
    for car_class in bookings_df["car_class"].unique():
        class_data = bookings_df[bookings_df["car_class"] == car_class]
        avg_price = class_data["booked_price"].mean()
        count = len(class_data)
        print(f"  {car_class}: ${avg_price:.2f} ({count:,} bookings)")
    
    # Supplier distribution
    print("\nBookings by supplier:")
    supplier_counts = bookings_df["supplier_id"].value_counts().sort_index()
    for supplier_id, count in supplier_counts.items():
        pct = count / len(bookings_df) * 100
        avg_price = bookings_df[bookings_df["supplier_id"] == supplier_id]["booked_price"].mean()
        print(f"  Supplier {supplier_id}: {count:,} ({pct:.1f}%) - Avg: ${avg_price:.2f}")
    
    # Booking timing
    print("\nBooking timing:")
    print(f"  Avg days before pickup: {bookings_df['days_until_pickup'].mean():.1f}")
    print(f"  Booking delay < 1 hour: {(bookings_df['booking_delay_hours'] < 1).sum() / len(bookings_df):.1%}")
    print(f"  Booking delay < 24 hours: {(bookings_df['booking_delay_hours'] < 24).sum() / len(bookings_df):.1%}")
    
    # Price competitiveness
    if "competitor_price" in bookings_df.columns and not bookings_df["competitor_price"].isna().all():
        price_diff = bookings_df["booked_price"] - bookings_df["competitor_price"]
        print("\nPrice competitiveness:")
        print(f"  Avg difference from competitor: ${price_diff.mean():.2f}")
        print(f"  Booked below competitor price: {(price_diff < 0).sum() / len(bookings_df):.1%}")



In [38]:
def generate_bookings_with_prices(
        searches_df: pd.DataFrame,
        rental_prices_df: pd.DataFrame,
        competitor_prices_df: Optional[pd.DataFrame] = None,
        segment_conversion_rates: dict[str, float] = segment_conversion_rates,
        expected_prices: dict[str, dict[str, float]] = expected_prices,
        price_sensitivity: dict[str, float] = price_sensitivity
    ) -> pd.DataFrame:
    """
    Generate bookings using actual rental price data.

    Args:
        searches_df: Search data with columns: search_id, user_id, location_id, 
                    car_class, search_ts, user_segment
        rental_prices_df: Rental price data with columns: location_id, supplier_id,
                            car_class, pickup_date, obs_ts, current_price
        competitor_prices_df: Optional competitor price data
        
    Returns:
        DataFrame with bookings including actual prices from rental_prices
    """
    print("Generating price-aware bookings...")

    # Validate inputs
    validate_inputs(searches_df, rental_prices_df)

    # Ensure datetime columns
    searches_df = searches_df.copy()
    rental_prices_df = rental_prices_df.copy()
    searches_df["search_ts"] = pd.to_datetime(searches_df["search_ts"])
    rental_prices_df["obs_ts"] = pd.to_datetime(rental_prices_df["obs_ts"])
    rental_prices_df["pickup_date"] = pd.to_datetime(rental_prices_df["pickup_date"])

    # Process each search
    bookings = []
    no_price_found = 0

    for _, search in searches_df.iterrows():
        # Find available prices for this search
        available_prices = find_available_prices(
            search, rental_prices_df, competitor_prices_df
        )
        
        if available_prices.empty:
            no_price_found += 1
            continue
        
        # Decide if this search converts to booking
        booking = attempt_booking(search, available_prices, segment_conversion_rates, expected_prices, price_sensitivity)
        if booking is not None:
            bookings.append(booking)
    
    if no_price_found > 0:
        print(f"Warning: {no_price_found} searches had no matching prices")
    
    # Create bookings DataFrame
    if bookings:
        bookings_df = pd.DataFrame(bookings)
        bookings_df = finalize_bookings(bookings_df, searches_df)
        
        print(f"Generated {len(bookings_df):,} bookings from {len(searches_df):,} searches")
        print(f"Conversion rate: {len(bookings_df)/len(searches_df):.2%}")
        
        print_booking_summary(bookings_df)
        return bookings_df
    else:
        print("No bookings generated")
        return pd.DataFrame()

In [39]:
bookings_df = generate_bookings_with_prices(
    searches_df,
    rental_prices_df,
    competitor_prices_df,
    segment_conversion_rates,
    expected_prices,
    price_sensitivity,
)

bookings_df.head()

Generating price-aware bookings...
Generated 3,205 bookings from 57,282 searches
Conversion rate: 5.60%

PRICE-AWARE BOOKING SUMMARY

Total bookings: 3,205

Price Statistics:
  Average booked price: $89.68
  Price range: $34.65 - $317.06

Average price by car class:
  luxury: $163.87 (575 bookings)
  compact: $66.05 (851 bookings)
  economy: $55.02 (894 bookings)
  suv: $99.22 (885 bookings)

Bookings by supplier:
  Supplier 1: 443 (13.8%) - Avg: $98.76
  Supplier 2: 357 (11.1%) - Avg: $96.31
  Supplier 3: 486 (15.2%) - Avg: $95.79
  Supplier 4: 580 (18.1%) - Avg: $89.25
  Supplier 5: 1,339 (41.8%) - Avg: $82.88

Booking timing:
  Avg days before pickup: 10.8
  Booking delay < 1 hour: 38.5%
  Booking delay < 24 hours: 70.5%

Price competitiveness:
  Avg difference from competitor: $-0.29
  Booked below competitor price: 33.5%


,booking_id,search_id,user_id,supplier_id,location_id,car_class,pickup_date,search_ts,booking_ts,booking_delay_hours,days_until_pickup,search_price,booked_price,competitor_price,price_rank,user_segment
0,1,981,19604,4,1,luxury,2024-01-07 00:00:00,2024-01-02 10:52:00,2024-01-02 11:10:14.743944,0.304096,4.547222,162.86,162.86,162.86,1,multi_trip
1,2,1010,16923,2,12,luxury,2024-01-11 03:00:00,2024-01-02 15:08:00,2024-01-02 15:18:42.738602,0.178539,8.494444,163.01,163.01,163.01,1,single_trip
2,3,1020,19731,5,23,compact,2024-01-05 02:00:00,2024-01-02 17:35:00,2024-01-02 18:02:24.496391,0.456805,2.350694,59.47,59.47,59.47,1,multi_trip
3,4,1027,19974,4,2,luxury,2024-01-13 20:00:00,2024-01-02 18:14:00,2024-01-03 09:37:31.975138,15.392215,11.073611,117.39,117.39,117.39,1,frequent_renter
4,5,1090,19834,2,10,economy,2024-01-06 14:00:00,2024-01-03 09:39:00,2024-01-03 10:01:43.756449,0.378821,3.181250,49.65,49.65,49.65,1,frequent_renter


## 💾 Save Generated Data

In [40]:
# Create output directory
os.makedirs(output_dir, exist_ok=True)

print("💾 Saving synthetic data to CSV files...")
print(f"📁 Output directory: {output_dir}/")


filename = os.path.join(output_dir, f"searches.csv")
searches_df.to_csv(filename, index=False)

filename = os.path.join(output_dir, f"users.csv")
users_df.to_csv(filename, index=False)

filename = os.path.join(output_dir, f"suppliers.csv")
suppliers_df.to_csv(filename, index=False)

filename = os.path.join(output_dir, f"car_classes.csv")
car_classes_df.to_csv(filename, index=False)

filename = os.path.join(output_dir, f"locations_weights.csv")
with open(filename.replace('.csv', '.json'), 'w') as f:
    json.dump(location_weights, f, indent=2)

filename = os.path.join(output_dir, f"bookings.csv")
bookings_df.to_csv(filename, index=False)

filename = os.path.join(output_dir, f"rental_prices.csv")
rental_prices_df.to_csv(filename, index=False)

filename = os.path.join(output_dir, f"competitor_prices.csv")
competitor_prices_df.to_csv(filename, index=False)


print(f"✅ Saved to {output_dir}/")


💾 Saving synthetic data to CSV files...
📁 Output directory: /Users/alejandro/workspace/car_rental_dynamic_book_prediction/data/sample/
✅ Saved to /Users/alejandro/workspace/car_rental_dynamic_book_prediction/data/sample/
